In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import splrep, splev
from scipy.integrate import quad
from sympy import init_printing, Symbol, Matrix, simplify, lambdify
init_printing()

In [ ]:
# geometry and material parameters
class geometry1D():
    def __init__(self, ne, deg):
        self.L = 1.e-4
        self.g = 9.81
        self.A = 10**-4*np.pi
        self.m = 320.
        self.C = 0.
        self.nNNodes=int(ne*deg+1)   #Isoparametric deg-th order lagrange elements
        self.ndim=1

class GPXi():
    def __init__(self, ordr):
        from numpy.polynomial.legendre import leggauss                         #Gauss-Legendre Quadrature for 1D
        self.xi=leggauss(ordr)[0]
        self.wght=leggauss(ordr)[1]

class basis():                                                                 # defined on the canonical element (1D : [-1,1] ) 
    def __init__(self,deg,basis_type):
        deg = int(deg)
        if basis_type == 'L':          # 1D Lagrange basis of degree `deg`
            z=Symbol('z')
            Xi=np.linspace(-1,1,deg+1)
            def lag_basis(k):
                n = 1.
                for i in range(len(Xi)):
                    if k != i:
                        n *= (z-Xi[i])/(Xi[k]-Xi[i])
                return n
            N = Array([simplify(lag_basis(m)) for m in range(deg+1)])            
            dfN = diff(N,z)+1.e-25*N
            self.Ns=lambdify(z, N, "numpy")
            self.dN=lambdify(z, dfN, "numpy")

def loc_mat(nodes,ElFlag,Ndnode):   #Computes the element quantities when supplied with gloabl nodes
    xi=GP.xi;W=GP.wght
    
    if etypes=='L':
        x = np.array(nB.Ns(xi)).T @ nodes
        Je = np.array(nB.dN(xi)).T @ nodes
        b1 = np.array(B.Ns(xi)).reshape(-1,1,W.size)
        a1 = np.array(B.dN(xi)).reshape(-1,1,W.size)  
        a2 = a1.reshape(1,len(a1),-1).copy()
        b2 = b1.reshape(1,len(b1),-1).copy()
        a1 *= E(x)/Je*geom.A*W       # multiply by weights, jacobian, A, in bitwise fashion 
        b1 *= geom.C*Je*W
        mat = np.tensordot(a1,a2,axes=([1,2],[0,2])) + np.tensordot(b1,b2,axes=([1,2],[0,2]))     #could potentially use einsum to clean it up (but works for now!)
        elemF = (np.array(B.Ns(xi))*W*Je*body_force(x)).sum(axis=1).flatten()
        return mat, elemF


def E(x):
    if 0. < x < 0.25 or 0.5 <= x < 0.75:
        return 200*10**9
    else:
        return 70*10**9

def rho(x):
    if 0. < x < 0.25 or 0.5 <= x < 0.75:
        return 7800.
    else:
        return 2700.

def f(x):
    if 0.375 <= x < 0.625:
        return 10**5*np.cos(4.*np.pi*(x-0.5))
    else:
        return 0.

def body_force(x):
    return rho(x)*geom.A*geom.g + f(x)


def uex(x):
    inner = 
        